# Deep Agent for Research: LangChain 기반 AI 리서치 에이전트 구축 튜토리얼

이 튜토리얼에서는 LangChain 및 LangGraph를 활용하여 효율적인 AI 리서치 에이전트(Deep Agent)를 구축하는 방법을 다룹니다. 웹 검색 결과를 파일로 저장하고, 요약 정보를 에이전트에게 전달하는 구조를 통해 토큰 효율성을 극대화합니다. 또한, 전략적 사고 도구와 서브에이전트(Research Sub-Agent) 위임 패턴을 적용하여 복잡한 리서치 워크플로우를 체계적으로 관리하는 방법을 배웁니다. 실습을 통해 직접 Deep Agent를 구현하고, `deepagents` 패키지를 활용한 간편한 추상화 방법도 소개합니다.

**🏆 주로 다루는 내용**
1. Deep Agent 아키텍처 개요  
   - 웹 검색 결과를 파일로 저장하고, 최소한의 요약 정보만 에이전트에게 전달하는 구조 및 토큰 효율성 문제 해결 방법을 설명합니다.
2. Tavily 기반 웹 검색 도구(tavily_search) 구현  
   - Tavily API를 활용하여 웹 검색을 수행하고, 결과를 파일로 저장하며, 요약 정보를 생성하는 핵심 함수 및 도구를 직접 구현합니다.
3. 웹페이지 내용 요약 및 파일 저장  
   - GPT-4o-mini 모델을 활용하여 웹페이지 내용을 구조적으로 요약하고, 파일명과 주요 학습 내용을 자동 생성하는 방법을 다룹니다.
4. 검색 결과 처리 및 에러 핸들링  
   - HTTP 요청을 통한 웹페이지 내용 수집, HTML을 Markdown으로 변환, 에러 발생 시 대체 요약 제공 등 검색 결과를 안전하게 처리하는 방법을 설명합니다.
5. 전략적 사고 도구(think_tool) 설계  
   - 리서치 진행 상황을 분석하고, 정보의 부족/충분 여부를 평가하며, 다음 단계 결정을 위한 전략적 사고 도구를 구현합니다.
6. Deep Agent 및 Research Sub-Agent 구성  
   - 메인 에이전트와 서브에이전트에 각각 적합한 도구와 프롬프트를 할당하여, 역할 분리와 위임 패턴을 적용하는 방법을 실습합니다.
7. Deep Agent 실습 및 응답 예시  
   - 실제로 에이전트에게 리서치 요청을 보내고, 응답 메시지를 포맷팅하여 결과를 확인하는 실습을 진행합니다.
8. deepagents 패키지 활용  
   - 직접 구현한 패턴을 기반으로, `deepagents` 패키지를 사용하여 Deep Agent를 간편하게 생성하는 방법을 소개합니다.

**✅ 주요 키워드**
- Deep Agent
- LangChain
- LangGraph
- Tavily API
- Research Sub-Agent
- File System Tools
- Summarization Model (GPT-4o-mini)
- think_tool
- Context Offloading
- Task Delegation

**⭐️ 사전 지식**
- Python 프로그래밍 기본 지식
- Jupyter Notebook 사용 경험
- LangChain 및 LangGraph 기본 개념 이해
- API 호출 및 HTTP 요청 처리 경험
- 인공지능 모델(특히 LLM) 활용 경험이 있으면 학습에 도움이 됩니다.

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# 환경 변수 로드
load_dotenv(override=True)
# 추적을 위한 프로젝트 이름 설정
logging.langsmith("Deep-Agent-Tutorial")

## Deep Agent for Research

### 개요

![agent_header](./assets/agent_header.png)

지금까지 학습한 내용을 바탕으로 실제 리서치 에이전트의 핵심 구조를 정리합니다.

- **TODO 리스트**를 활용하여 각 작업의 진행 상황을 체계적으로 관리합니다.
- **파일 시스템**을 이용해 도구 실행 결과(원본 데이터)를 저장합니다.
- **서브에이전트**에게 리서치 작업을 위임하여, 각 에이전트의 컨텍스트를 분리하고 효율적으로 관리합니다.

---

### Search Tool 설계

검색 도구는 웹에서 수집한 원본 데이터를 파일로 저장하고, 에이전트에게는 요약 정보만 전달하는 구조로 설계합니다. 이 방식은 장기적인 에이전트 워크플로우에서 매우 효과적이며, 실제 사례로는 [Manus의 Context Engineering](https://manus.im/blog/Context-Engineering-for-AI-Agents-Lessons-from-Building-Manus)에서 확인할 수 있습니다.

#### 핵심 구성 요소

1. **검색 실행 (`run_tavily_search`)**  
   Tavily API를 활용하여 실제 웹 검색을 수행합니다. 결과 개수 및 주제 필터링 등 다양한 파라미터를 설정할 수 있습니다.

2. **콘텐츠 요약 (`summarize_webpage_content`)**  
   `GPT-4o-mini` 모델을 사용하여 웹페이지의 주요 내용을 구조적으로 요약합니다. 요약 결과에는 파일명과 핵심 학습 내용이 포함됩니다.

3. **결과 처리 (`process_search_results`)**  
   HTTP를 통해 웹페이지 전체 내용을 수집하고, `markdownify`를 이용해 HTML을 Markdown으로 변환합니다. 각 결과에 대해 요약 정보를 생성합니다.

4. **컨텍스트 오프로딩 (`tavily_search` tool)**  
   검색 및 결과 처리를 실행하고, 전체 원본 데이터는 파일로 저장합니다. 에이전트에게는 최소한의 요약 정보만 반환하여 불필요한 컨텍스트 확장을 방지합니다. 또한, LangGraph의 `Command`를 활용해 파일과 메시지를 동시에 관리합니다.

5. **전략적 사고 도구 (`think_tool`)**  
   에이전트가 리서치 결과를 분석하고, 정보의 부족/충분 여부를 평가하며, 다음 단계의 계획을 세울 수 있도록 구조화된 사고 프레임워크를 제공합니다.

이러한 아키텍처는 상세한 검색 결과를 파일로 분리 저장함으로써, 에이전트의 컨텍스트를 최소화하고 토큰 효율성을 극대화할 수 있습니다.

> **참고**  
`create_react_agent` 함수는 LangGraph 라이브러리에서 LangChain 라이브러리로 이동되었으며, 1.0 버전 코드 릴리스 이후에는 `create_agent`로 이름이 변경되었습니다. 이에 따라 import 및 코드 사용 방식에 약간의 변경이 있으니 참고하시기 바랍니다. 영상에서는 이전 설정이 사용될 수 있습니다. 최신 코드 예제에서는 변경된 사항을 반영해 주시기 바랍니다.

In [ ]:
%%writefile ../src/deep_agents_from_scratch/research_tools.py
"""Research Tools.

This module provides search and content processing utilities for the research agent,
including web search capabilities and content summarization tools.
"""
import os
from datetime import datetime
import uuid, base64

import httpx
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import InjectedToolArg, InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState
from langgraph.types import Command
from markdownify import markdownify
from pydantic import BaseModel, Field
from tavily import TavilyClient
from typing_extensions import Annotated, Literal

from deep_agents_from_scratch.prompts import SUMMARIZE_WEB_SEARCH
from deep_agents_from_scratch.state import DeepAgentState

# 요약 모델 초기화 (GPT-4o-mini) 및 Tavily API 클라이언트 생성
summarization_model = init_chat_model(model="openai:gpt-4o-mini")
tavily_client = TavilyClient()

class Summary(BaseModel):
    """Schema for webpage content summarization."""
    # 요약 결과 파일명 및 주요 학습 내용 저장을 위한 필드 정의
    filename: str = Field(description="Name of the file to store.")
    summary: str = Field(description="Key learnings from the webpage.")

# 현재 날짜를 사람이 읽기 쉬운 문자열로 반환
def get_today_str() -> str:
    """Get current date in a human-readable format."""
    return datetime.now().strftime("%a %b %-d, %Y")

# Tavily API를 활용한 단일 쿼리 웹 검색 실행
def run_tavily_search(
    search_query: str,
    max_results: int = 1,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = True,
) -> dict:
    """Perform search using Tavily API for a single query.

    Args:
        search_query: Search query to execute
        max_results: Maximum number of results per query
        topic: Topic filter for search results
        include_raw_content: Whether to include raw webpage content

    Returns:
        Search results dictionary
    """
    result = tavily_client.search(
        search_query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic
    )

    return result

# 웹페이지 원본 내용을 요약 모델로 구조적 요약 및 파일명 생성
def summarize_webpage_content(webpage_content: str) -> Summary:
    """Summarize webpage content using the configured summarization model.

    Args:
        webpage_content: Raw webpage content to summarize

    Returns:
        Summary object with filename and summary
    """
    try:
        # Structured output 모델을 활용한 요약 결과 생성
        structured_model = summarization_model.with_structured_output(Summary)

        # HumanMessage 프롬프트로 요약 및 파일명 동시 생성
        summary_and_filename = structured_model.invoke([
            HumanMessage(content=SUMMARIZE_WEB_SEARCH.format(
                webpage_content=webpage_content,
                date=get_today_str()
            ))
        ])

        return summary_and_filename

    except Exception:
        # 요약 실패 시 기본 파일명 및 앞부분 요약 반환
        return Summary(
            filename="search_result.md",
            summary=webpage_content[:1000] + "..." if len(webpage_content) > 1000 else webpage_content
        )

# Tavily 검색 결과를 받아 각 결과에 대해 요약 및 파일명 생성, 에러 처리 포함
def process_search_results(results: dict) -> list[dict]:
    """Process search results by summarizing content where available.

    Args:
        results: Tavily search results dictionary

    Returns:
        List of processed results with summaries
    """
    processed_results = []

    # HTTP 요청용 클라이언트 생성 및 30초 타임아웃 설정
    HTTPX_CLIENT = httpx.Client(timeout=30.0)  # Add 30 second timeout

    for result in results.get('results', []):

        # 결과 URL 추출
        url = result['url']

        # URL 접속 및 에러 핸들링, HTML을 Markdown으로 변환
        try:
            response = HTTPX_CLIENT.get(url)

            if response.status_code == 200:
                # HTML 원문을 Markdown으로 변환 후 요약
                raw_content = markdownify(response.text)
                summary_obj = summarize_webpage_content(raw_content)
            else:
                # Tavily가 제공한 요약 및 원본 내용 사용
                raw_content = result.get('raw_content', '')
                summary_obj = Summary(
                    filename="URL_error.md",
                    summary=result.get('content', 'Error reading URL; try another search.')
                )
        except (httpx.TimeoutException, httpx.RequestError) as e:
            # 타임아웃 또는 연결 오류 발생 시 대체 요약 및 파일명 지정
            raw_content = result.get('raw_content', '')
            summary_obj = Summary(
                filename="connection_error.md",
                summary=result.get('content', f'Could not fetch URL (timeout/connection error). Try another search.')
            )

        # 파일명 중복 방지를 위한 UUID 기반 고유값 생성 및 파일명에 추가
        uid = base64.urlsafe_b64encode(uuid.uuid4().bytes).rstrip(b"=").decode("ascii")[:8]
        name, ext = os.path.splitext(summary_obj.filename)
        summary_obj.filename = f"{name}_{uid}{ext}"

        # 최종 결과 리스트에 요약, 파일명, 원본 내용 등 저장
        processed_results.append({
            'url': result['url'],
            'title': result['title'],
            'summary': summary_obj.summary,
            'filename': summary_obj.filename,
            'raw_content': raw_content,
        })

    return processed_results

# Tavily 검색 및 결과 파일 저장, 에이전트에게 최소 요약 정보만 반환하는 도구 함수
@tool(parse_docstring=True)
def tavily_search(
    query: str,
    state: Annotated[DeepAgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId],
    max_results: Annotated[int, InjectedToolArg] = 1,
    topic: Annotated[Literal["general", "news", "finance"], InjectedToolArg] = "general",
) -> Command:
    """Search web and save detailed results to files while returning minimal context.

    Performs web search and saves full content to files for context offloading.
    Returns only essential information to help the agent decide on next steps.

    Args:
        query: Search query to execute
        state: Injected agent state for file storage
        tool_call_id: Injected tool call identifier
        max_results: Maximum number of results to return (default: 1)
        topic: Topic filter - 'general', 'news', or 'finance' (default: 'general')

    Returns:
        Command that saves full results to files and provides minimal summary
    """
    # Tavily 검색 실행 및 원본 포함 결과 반환
    search_results = run_tavily_search(
        query,
        max_results=max_results,
        topic=topic,
        include_raw_content=True,
    )

    # 검색 결과 요약 및 파일명 생성
    processed_results = process_search_results(search_results)

    # 파일 시스템에 결과 저장 및 요약 리스트 생성
    files = state.get("files", {})
    saved_files = []
    summaries = []

    for i, result in enumerate(processed_results):
        # 요약 모델에서 생성된 파일명 사용
        filename = result['filename']

        # 파일 내용: 제목, URL, 쿼리, 날짜, 요약, 원본 내용 포함
        file_content = f"""# Search Result: {result['title']}

**URL:** {result['url']}
**Query:** {query}
**Date:** {get_today_str()}

## Summary
{result['summary']}

## Raw Content
{result['raw_content'] if result['raw_content'] else 'No raw content available'}
"""

        files[filename] = file_content
        saved_files.append(filename)
        summaries.append(f"- {filename}: {result['summary']}...")

    # 에이전트에게 반환할 최소 요약 메시지 생성 (파일명 및 요약 리스트)
    summary_text = f"""🔍 Found {len(processed_results)} result(s) for '{query}':

{chr(10).join(summaries)}

Files: {', '.join(saved_files)}
💡 Use read_file() to access full details when needed."""

    return Command(
        update={
            "files": files,
            "messages": [
                ToolMessage(summary_text, tool_call_id=tool_call_id)
            ],
        }
    )

# 리서치 진행 상황 분석 및 전략적 판단을 위한 사고 도구 함수
@tool(parse_docstring=True)
def think_tool(reflection: str) -> str:
    """Tool for strategic reflection on research progress and decision-making.

    Use this tool after each search to analyze results and plan next steps systematically.
    This creates a deliberate pause in the research workflow for quality decision-making.

    When to use:
    - After receiving search results: What key information did I find?
    - Before deciding next steps: Do I have enough to answer comprehensively?
    - When assessing research gaps: What specific information am I still missing?
    - Before concluding research: Can I provide a complete answer now?
    - How complex is the question: Have I reached the number of search limits?

    Reflection should address:
    1. Analysis of current findings - What concrete information have I gathered?
    2. Gap assessment - What crucial information is still missing?
    3. Quality evaluation - Do I have sufficient evidence/examples for a good answer?
    4. Strategic decision - Should I continue searching or provide my answer?

    Args:
        reflection: Your detailed reflection on research progress, findings, gaps, and next steps

    Returns:
        Confirmation that reflection was recorded for decision-making
    """
    return f"Reflection recorded: {reflection}"

## Deep Agent

지금까지 학습한 내용을 바탕으로, 실제 리서치 에이전트의 전체 구조를 적용할 수 있습니다.

- `think_tool`과 앞서 구현한 `search_tool`을 리서치 서브에이전트에게 할당하여, 정보 탐색과 전략적 사고를 효과적으로 수행할 수 있도록 구성합니다.
- 상위 에이전트(Parent Agent)에는 파일 시스템 도구, `think_tool`, 그리고 `task` 도구를 함께 제공하여, 역할 분리와 효율적인 작업 위임이 가능하도록 설계합니다.

이와 같은 구조를 통해 각 에이전트가 자신의 역할에 집중할 수 있으며, 복잡한 리서치 워크플로우도 체계적으로 관리할 수 있습니다.

In [ ]:
from datetime import datetime

from IPython.display import Image, display
from langchain.chat_models import init_chat_model

# from langgraph.prebuilt import create_react_agent
from langchain.agents import create_agent
from utils import show_prompt, stream_agent

from deep_agents_from_scratch.file_tools import ls, read_file, write_file
from deep_agents_from_scratch.prompts import (
    FILE_USAGE_INSTRUCTIONS,
    RESEARCHER_INSTRUCTIONS,
    SUBAGENT_USAGE_INSTRUCTIONS,
    TODO_USAGE_INSTRUCTIONS,
)
from deep_agents_from_scratch.research_tools import (
    tavily_search,
    think_tool,
    get_today_str,
)
from deep_agents_from_scratch.state import DeepAgentState
from deep_agents_from_scratch.task_tool import _create_task_tool
from deep_agents_from_scratch.todo_tools import write_todos, read_todos

# 기본 LLM 모델 초기화 및 온도 설정 (Anthropic Claude Sonnet 4 사용)
model = init_chat_model(model="anthropic:claude-sonnet-4-20250514", temperature=0.0)

# 리서치 서브에이전트 동시 실행 최대 개수 및 반복 횟수 제한 설정
max_concurrent_research_units = 3
max_researcher_iterations = 3

# 리서치 서브에이전트에 할당할 도구 리스트 (웹 검색 및 전략적 사고 도구)
sub_agent_tools = [tavily_search, think_tool]
# 메인 에이전트에 할당할 파일 시스템 및 TODO 관리 도구, 전략적 사고 도구 포함
built_in_tools = [ls, read_file, write_file, write_todos, read_todos, think_tool]

# 리서치 서브에이전트 구성 정보 (이름, 설명, 프롬프트, 도구 목록)
research_sub_agent = {
    "name": "research-agent",
    "description": "Delegate research to the sub-agent researcher. Only give this researcher one topic at a time.",
    "prompt": RESEARCHER_INSTRUCTIONS.format(date=get_today_str()),
    "tools": ["tavily_search", "think_tool"],
}

# 서브에이전트에게 작업 위임을 위한 Task Tool 생성 (모델, 상태, 도구 전달)
task_tool = _create_task_tool(
    sub_agent_tools, [research_sub_agent], model, DeepAgentState
)

# 메인 에이전트에 위임 도구 포함 전체 도구 리스트 생성 (검색 도구 직접 사용 가능)
delegation_tools = [task_tool]
all_tools = (
    sub_agent_tools + built_in_tools + delegation_tools
)  # search available to main agent for trivial cases

# 서브에이전트 프롬프트 생성 (최대 동시 리서치 개수, 반복 횟수, 날짜 포함)
SUBAGENT_INSTRUCTIONS = SUBAGENT_USAGE_INSTRUCTIONS.format(
    max_concurrent_research_units=max_concurrent_research_units,
    max_researcher_iterations=max_researcher_iterations,
    date=datetime.now().strftime("%a %b %-d, %Y"),
)

In [ ]:
# 리서처 에이전트 프롬프트 출력
show_prompt(RESEARCHER_INSTRUCTIONS)

In [ ]:
# TODO 관리, 파일 시스템 사용, 서브에이전트 위임 관련 프롬프트를 하나의 문자열로 통합
INSTRUCTIONS = (
    "# TODO MANAGEMENT\n"
    + TODO_USAGE_INSTRUCTIONS
    + "\n\n"
    + "=" * 80
    + "\n\n"
    + "# FILE SYSTEM USAGE\n"
    + FILE_USAGE_INSTRUCTIONS
    + "\n\n"
    + "=" * 80
    + "\n\n"
    + "# SUB-AGENT DELEGATION\n"
    + SUBAGENT_INSTRUCTIONS
)

# 통합된 프롬프트를 화면에 출력하여 에이전트의 역할 및 사용법 안내
show_prompt(INSTRUCTIONS)

In [ ]:
# Deep Agent 생성 및 설정, 시스템 프롬프트와 상태 스키마 적용
agent = create_agent(  # updated 1.0
    model, all_tools, system_prompt=INSTRUCTIONS, state_schema=DeepAgentState
)

# 에이전트의 내부 워크플로우 구조 시각화, Mermaid PNG로 출력
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
from utils import format_messages

# 에이전트에게 MCP 개요 요청 메시지 전달 및 응답 결과 저장
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Give me an overview of Model Context Protocol (MCP).",
            }
        ],
    }
)

# 에이전트 응답 메시지 포맷팅 및 출력, 사용자 친화적 결과 확인 목적
format_messages(result["messages"])

**실행 결과 예시 및 참고 자료**

아래 링크에서는 본 튜토리얼에서 구현한 Deep Agent의 실제 실행 결과와 워크플로우를 확인할 수 있습니다.  
에이전트의 동작 방식, 검색 및 요약 처리, 파일 저장 등 전체 프로세스가 LangChain Smith 플랫폼에서 시각적으로 제공됩니다.

- [실행 트레이스 예시 보기](https://smith.langchain.com/public/3a389ec6-8e6e-4f9e-9a82-0d0a9569e6f8/r)

이 자료를 참고하면, 본 튜토리얼에서 다루는 Deep Agent의 구조와 각 단계별 동작을 더욱 명확하게 이해할 수 있습니다.  
실제 에이전트의 처리 흐름과 결과물을 직접 확인하며, 실습 과정에서 발생할 수 있는 다양한 상황에 대한 이해도를 높일 수 있습니다.

## deepagents 패키지 활용하기

지금까지 Deep Agent의 핵심 패턴과 구조를 직접 구현하며 학습하였습니다.  
이제는 이러한 패턴을 간편하게 사용할 수 있도록 추상화된 [`deepagents` 패키지](https://github.com/hwchase17/deepagents)를 활용할 수 있습니다.

`deepagents` 패키지는 다음과 같은 주요 기능을 기본적으로 제공합니다.

- 파일 시스템 도구(file system tools) 내장
- TODO 관리 도구(todo tool) 포함
- 작업 위임 도구(task tool) 제공

사용자는 서브에이전트(sub-agent)와 해당 서브에이전트가 사용할 도구만 지정하면, 복잡한 리서치 에이전트 구조를 손쉽게 구축할 수 있습니다.

이처럼 `deepagents` 패키지를 활용하면, 앞서 배운 구조와 워크플로우를 더욱 빠르고 효율적으로 적용할 수 있습니다.  
직접 구현한 패턴을 기반으로, 실무 환경에서도 신속하게 Deep Agent를 생성하고 운영할 수 있습니다.

In [ ]:
# deepagents 패키지 기반 Deep Agent 생성 및 설정
from deepagents import create_deep_agent

# 촬영 이후 코드 업데이트 반영

# 리서치 서브에이전트 구성 정보, 시스템 프롬프트 및 도구 할당
research_sub_agent_2 = {
    "name": "research-agent",
    "description": "Delegate research to the sub-agent researcher. Only give this researcher one topic at a time.",
    "system_prompt": RESEARCHER_INSTRUCTIONS.format(date=get_today_str()),
    "tools": [tavily_search, think_tool],
}

# Deep Agent 생성, 전체 도구, 시스템 프롬프트, 서브에이전트, 모델 전달
agent = create_deep_agent(  # 업데이트된 함수 사용
    tools=sub_agent_tools,
    system_prompt=INSTRUCTIONS,
    subagents=[research_sub_agent_2],
    model=model,
)

# 에이전트 내부 워크플로우 구조 시각화 코드 (주석 처리됨)
# display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# 사용자 메시지로 MCP에 대한 매우 간단한 개요 요청
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Give me an very brief overview of Model Context Protocol (MCP).",
            }
        ],
    }
)

# 에이전트 응답 메시지 포맷팅 및 결과 출력
format_messages(result["messages"])

실제 Deep Agent의 실행 결과와 워크플로우를 확인하고 싶으신 경우, 아래 링크를 참고하실 수 있습니다.

- [실행 트레이스 예시 보기](https://smith.langchain.com/public/1d626d81-a102-4588-a2fb-cab40a7271f1/r)

이 자료에서는 본 튜토리얼에서 구현한 Deep Agent가 어떻게 검색, 요약, 파일 저장 등 각 단계를 처리하는지 LangChain Smith 플랫폼을 통해 시각적으로 확인할 수 있습니다.  
실제 에이전트의 동작 흐름과 결과물을 직접 살펴보면, 복잡한 리서치 워크플로우의 구조와 각 단계별 처리 과정을 더욱 명확하게 이해할 수 있습니다.

실습 과정에서 발생할 수 있는 다양한 상황에 대한 이해도를 높이고, Deep Agent의 전체적인 구조와 응답 방식에 대한 실질적인 예시를 제공하므로, 학습에 적극적으로 활용하시기 바랍니다.

In [ ]:
# Deep Agent 생성 및 설정, 시스템 프롬프트와 상태 스키마 적용
agent = create_agent(  # updated 1.0
    model, all_tools, system_prompt=INSTRUCTIONS, state_schema=DeepAgentState
)

# 에이전트의 내부 워크플로우 구조 시각화, Mermaid PNG로 출력
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))